In [1]:
# Importando bibliotecas
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
from pandas.tseries.offsets import MonthEnd
from functions import *
from FUNCTIONS_LTP import *
import locale
from openpyxl import load_workbook
from openpyxl.utils import range_boundaries
import tempfile
import shutil
import gc
from collections import defaultdict

locale.setlocale(locale.LC_TIME, 'Portuguese_Brazil.1252')  # Para Windows
timer = Temporizador()
timer.iniciar()

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Aumenta o limite de largura da coluna para exibição
pd.set_option('display.expand_frame_repr', False)

# Detecta se o script está sendo executado de um .py ou de um notebook
try:
    caminho_base = Path(__file__).resolve().parent
except NameError:
    # __file__ não existe em Jupyter ou ambiente interativo
    caminho_base = Path.cwd()
    
pasta_input = caminho_base.parent / '01_INPUT'
pasta_output = caminho_base.parent / '02_OUTPUT'
pasta_painel = caminho_base.parent / '03_EXCEL'

# Carregar bd_mat_param uma vez
bd_mat_param = pd.read_excel(
    pasta_input / 'matriz_parametros.xlsx',
    sheet_name='matriz_parametros',
    engine='calamine',
    dtype={'produto': str}
)
bd_mat_param['produto'] = bd_mat_param['produto'].astype(str).str.strip().str.upper()
produtos_param = set(bd_mat_param['produto'])
colunas_manter = [
    'produto', 'unidade_fat', 'unidade_prod', 'tipo_abast', 'prioridade'
]

bd_mat_param = bd_mat_param[colunas_manter].reset_index(drop=True)

# Carregar base_dados_produtos
bd_prod = pd.read_excel(
    pasta_input / 'base_dados_produtos.xlsx',
    sheet_name='base_dados_produtos',
    engine='calamine',
    dtype={'cod_produto': str}
)
bd_prod['cod_produto'] = bd_prod['cod_produto'].astype(str).str.strip().str.upper()
bd_prod['cod_produto'] = bd_prod['cod_produto'].astype(str).str.strip().str.upper()
bd_prod = bd_prod[bd_prod['cod_produto'].isin(produtos_param)].copy().reset_index(drop=True)
if 'mes_ref' in bd_prod.columns:
    bd_prod['mes_ref'] = pd.to_datetime(bd_prod['mes_ref'], errors='coerce', dayfirst=True)

colunas_manter = ['mes_ref', 'empresa', 'cod_produto', 'descricao', 'linha_prod', 'familia_prod', 'tipo_produto', 'curva_abc', 'curva_123', 'estoq_seg_pcs', 'estoq_seg_kg', 'estoq_inicial_pcs', 'estoq_inicial_kg', 'carteira_arraste_mes_anterior', 'carteira_mes_atual', 'previsao_pcs', 'saldo_previsao_pcs', 'peso_produto_kg', 'estoq_transf_pcs', 'lote_econ', 'qtd_emb', 'lote_min']
bd_prod = bd_prod[colunas_manter].reset_index(drop=True)

# Carregar estruturas
colunas_bd_estruturas = ['mes_ref', 'empresa', 'cod_prod_acabado', 'cod_insumo', 'descricao', 'qtd_utilizada_pcs']
bd_estruturas = pd.read_excel(
    pasta_input / 'estruturas.xlsx',
    sheet_name='estruturas',
    engine='calamine',
    dtype={'cod_prod_acabado': str}
)
bd_estruturas['cod_prod_acabado'] = bd_estruturas['cod_prod_acabado'].astype(str).str.strip().str.upper()
bd_estruturas['cod_insumo'] = bd_estruturas['cod_insumo'].astype(str).str.strip().str.upper()
bd_estruturas = bd_estruturas[bd_estruturas['cod_prod_acabado'].isin(produtos_param)].copy().reset_index(drop=True)
bd_estruturas = bd_estruturas[colunas_bd_estruturas]
# Elminar coluna mes_ref e remover duplicatas
bd_estruturas = bd_estruturas.drop(columns=['mes_ref'])
bd_estruturas = bd_estruturas.drop_duplicates(subset=['empresa', 'cod_prod_acabado', 'cod_insumo']).reset_index(drop=True)

print("✅ Mapeamento de pastas e importação de tabelas concluído!")

✅ Mapeamento de pastas e importação de tabelas concluído!


In [2]:
# Gerando a tabela Calendário

# Definir intervalo de datas: do primeiro mês ao último mês completo
data_inicial = bd_prod['mes_ref'].min().replace(day=1)
data_final = bd_prod['mes_ref'].max() + MonthEnd(1)

# Gerar datas diariamente entre data_inicial e data_final
datas = pd.date_range(start=data_inicial, end=data_final, freq='D')

# Mapeamento dos dias da semana
dias_semana = {0: 'SEG', 1: 'TER', 2: 'QUA', 3: 'QUI', 4: 'SEX', 5: 'SÁB', 6: 'DOM'}

# Construção do DataFrame
df_calendario = pd.DataFrame({
    'data_calend': datas,
    'mes_calend': datas.to_series().apply(lambda d: d.replace(day=1)),
    'dia_calend': datas.to_series().dt.weekday.map(dias_semana)
})

# Importar aba Dia_Semana da planilha dados_calendario, com dados por dia da semana x Unidade
df_dia_semana = pd.read_excel(pasta_input / 'dados_calendario.xlsx', engine='calamine', sheet_name='Dia_Semana')

# Normalização (unpivot)
bd_dia_semana = df_dia_semana.melt(
    id_vars=['UNIDADE'],
    var_name='DIA_DE_SEMANA',
    value_name='OCUPACAO'
)

# Remove linhas onde OCUPACAO está vazia ou inválida (ex: '-')
bd_dia_semana = bd_dia_semana[bd_dia_semana['OCUPACAO'].notna()]
bd_dia_semana = bd_dia_semana[bd_dia_semana['OCUPACAO'] != '-']
bd_dia_semana['OCUPACAO'] = bd_dia_semana['OCUPACAO'].astype(float)

# Fazendo o merge entre o calendário e os dados de ocupação por unidade/dia da semana
df_calendario_ocup = df_calendario.merge(
    bd_dia_semana,
    left_on='dia_calend',
    right_on='DIA_DE_SEMANA',
    how='left'
)

# Selecionar apenas as colunas desejadas
df_calendario_ocup = df_calendario_ocup[
    ['data_calend', 'mes_calend', 'dia_calend', 'UNIDADE', 'OCUPACAO']
]

# Renomear colunas para maiúsculas
df_calendario_ocup.columns = [col.upper() for col in df_calendario_ocup.columns]

# Importar aba Feriados da planilha dados_calendario, com dados por dia da semana x Unidade
df_feriados = pd.read_excel(pasta_input / 'dados_calendario.xlsx', engine='calamine', sheet_name='Feriados')

# Garantir que as datas estejam no mesmo formato
df_feriados['FERIADO'] = pd.to_datetime(df_feriados['FERIADO'], format="%d/%m/%Y")

# Realizar o merge
df_calendario_ocup = df_calendario_ocup.merge(
    df_feriados[['FERIADO', 'UNIDADE', 'TIPO', 'NORMAL', 'REVEZAMENTO']],
    left_on=['DATA_CALEND', 'UNIDADE'],
    right_on=['FERIADO', 'UNIDADE'],
    how='left'
)

data_agora = datetime.now()
# Inserindo coluna data e hora agora
df_calendario_ocup['DATA_AGORA'] = data_agora

# NOVO_NORMAL: Se TIPO for NaN, retorna OCUPACAO; senão, retorna TIPO NORMAL
df_calendario_ocup['NOVO_NORMAL'] = df_calendario_ocup.apply(lambda row: row['OCUPACAO'] if pd.isna(row['TIPO']) else row['NORMAL'],axis=1)

# NOVO_REVEZAMENTO: Se TIPO for NaN, retorna 1; senão, retorna TIPO REVEZAMENTO
df_calendario_ocup['NOVO_REVEZAMENTO'] = df_calendario_ocup.apply(lambda row: 1 if pd.isna(row['TIPO']) else row['REVEZAMENTO'],axis=1)

# Coluna DIAS_NOR_CALEND: se DATA_CALEND <= DATA_AGORA retorna 0, senão NOVO_NORMAL
df_calendario_ocup['DIAS_NOR_CALEND_'] = df_calendario_ocup.apply(lambda row: 0 if row['DATA_CALEND'] <= row['DATA_AGORA'] else row['NOVO_NORMAL'],axis=1)

# Coluna DIAS_REV_CALEND: se DATA_CALEND <= DATA_AGORA retorna 0, senão NOVO_REVEZAMENTO
df_calendario_ocup['DIAS_REV_CALEND_'] = df_calendario_ocup.apply(lambda row: 0 if row['DATA_CALEND'] <= row['DATA_AGORA'] else row['NOVO_REVEZAMENTO'],axis=1)

# Coluna PARCIAL_HOJE: se DATA_CALEND = DATA_AGORA, RETORNA 1 - (Hora Agora / 24)
hora_agora = data_agora.hour + data_agora.minute / 60 + data_agora.second / 3600
data_hoje = data_agora.date()

df_calendario_ocup['PARCIAL_HOJE'] = df_calendario_ocup['DATA_CALEND'].dt.date.apply(lambda d: 1 - (hora_agora / 24) if d == data_hoje else 0)

df_calendario_ocup['DIAS_NOR_CALEND'] = df_calendario_ocup['DIAS_NOR_CALEND_'] + df_calendario_ocup['PARCIAL_HOJE']
df_calendario_ocup['DIAS_REV_CALEND'] = df_calendario_ocup['DIAS_REV_CALEND_'] + df_calendario_ocup['PARCIAL_HOJE']

bd_calend = df_calendario_ocup.groupby(['MES_CALEND', 'UNIDADE'], as_index=False)[['DIAS_NOR_CALEND', 'DIAS_REV_CALEND']].sum()
bd_calend = bd_calend.sort_values(by=['MES_CALEND', 'UNIDADE'])
bd_calend = bd_calend.rename(columns={
    'DIAS_NOR_CALEND': 'TOT_DIAS_NOR_CALEND',
    'DIAS_REV_CALEND': 'TOT_DIAS_REV_CALEND'
})

bd_calend['TOT_HORAS_NOR_CALEND'] = bd_calend['TOT_DIAS_NOR_CALEND'] * 24
bd_calend['TOT_HORAS_REV_CALEND'] = bd_calend['TOT_DIAS_REV_CALEND'] * 24

# del bd_dia_semana
# del df_calendario_ocup
# del df_calendario
# del df_feriados
# gc.collect()

print("✅ Geração de tabelas de calendário concluído!")

✅ Geração de tabelas de calendário concluído!


In [3]:
# Inicio Step = GERAR DADOS LTP
bd_prod['NEC_BASE_PROD_PCS'] = (
    bd_prod['estoq_inicial_pcs'] 
    - bd_prod['carteira_mes_atual'] 
    - bd_prod['carteira_arraste_mes_anterior'] 
    - bd_prod['previsao_pcs'] 
    - bd_prod['estoq_seg_pcs']
)

bd_prod['PRIORIDADE'] = 1

bd_mat_param_prioridade_1 = bd_mat_param[bd_mat_param['prioridade'] == 1]

# Join completo entre bd_prod e bd_mat_param
bd_prod_nec = pd.merge(
    bd_mat_param_prioridade_1,
    bd_prod,
    left_on=['produto', 'unidade_fat', 'prioridade'],
    right_on=['cod_produto', 'empresa', 'PRIORIDADE'],
    how='outer'  # FULL OUTER JOIN
)

# Importar aba matriz_regioes da planilha matriz regioes
bd_matriz_regioes = pd.read_excel(pasta_input / 'matriz_regioes.xlsx', sheet_name='matriz_regioes', engine='calamine')

# Merge para adicionar a Região  de Faturamento na base de produtos
bd_prod_nec = bd_prod_nec.merge(
    bd_matriz_regioes,
    how='left',
    left_on='empresa',
    right_on='Unidade'
)

bd_prod_nec = bd_prod_nec.rename(columns={'Unidade': 'Unidade_Fat'})
bd_prod_nec = bd_prod_nec.rename(columns={'Região': 'Reg_Unid_Fat'})

# Merge para adicionar a Região Unidade Produtiva na base de produtos
bd_prod_nec = bd_prod_nec.merge(
    bd_matriz_regioes,
    how='left',
    left_on='unidade_prod',
    right_on='Unidade'
)

bd_prod_nec = bd_prod_nec.rename(columns={'Unidade': 'Unidade_Prod'})
bd_prod_nec = bd_prod_nec.rename(columns={'Região': 'Reg_Unid_Prod'})

# Criando a coluna MESMA REGIAO, com os critérios aplicados
bd_prod_nec['MESMA_REG'] = bd_prod_nec.apply(
    lambda row: "NAO" if pd.isna(row['Reg_Unid_Prod']) else ("SIM" if row['Reg_Unid_Fat'] == row['Reg_Unid_Prod'] else "NAO"),
    axis=1
)

# Inserindo a coluna Mes_Ref_Ant na bd_prod, e criando uma bd_prod_mes_anterior
bd_prod_mes_anterior = bd_prod[['mes_ref', 'empresa', 'cod_produto', 'descricao', 'saldo_previsao_pcs']].copy()
bd_prod_mes_anterior = bd_prod_mes_anterior.rename(columns=
    {'mes_ref': 'MES_REF', 
     'empresa': 'EMPRESA', 
     'cod_produto': 'COD_PROD', 
     'descricao': 'DESC_PROD',
     'saldo_previsao_pcs': 'SALDO_PREV_PROX_MES_PCS'
     }
)

# Criando coluna MES_REF_ANT para definir mes do saldo da previsão
bd_prod_mes_anterior['MES_REF_ANT'] = (bd_prod_mes_anterior['MES_REF'] - pd.DateOffset(months=1)).dt.to_period('M').dt.to_timestamp()

# Adicionar a coluna SALDO_PREV_PROX_MES_PCS na bd_prod_nec
# Criando um dicionário com as chaves de busca e a coluna que você quer trazer
map_dict = bd_prod_mes_anterior.set_index(['MES_REF_ANT', 'EMPRESA', 'COD_PROD'])['SALDO_PREV_PROX_MES_PCS'].to_dict()

# Usando map para trazer a coluna 'SALDO_PREV_PROX_MES_PCS' para bd_prod_nec
bd_prod_nec['SALDO_PREV_PROX_MES_PCS'] = bd_prod_nec[['mes_ref', 'empresa', 'cod_produto']].apply(
    lambda row: map_dict.get((row['mes_ref'], row['empresa'], row['cod_produto']), None), axis=1
)

# Eliminando linhas filtrando mes_ref que não seja null
bd_prod_nec = bd_prod_nec[bd_prod_nec['mes_ref'].notna()]

# Criando a coluna PCS_NEC_PROD_MESMA_REG_SIM_NAO
# Preencher todos os NaN/None das colunas usadas no cálculo com zero
colunas_nec = [
    'carteira_arraste_mes_anterior', 'carteira_mes_atual', 'saldo_previsao_pcs',
    'SALDO_PREV_PROX_MES_PCS', 'estoq_seg_pcs', 'estoq_inicial_pcs', 'estoq_transf_pcs'
]
for col in colunas_nec:
    bd_prod_nec[col] = pd.to_numeric(bd_prod_nec[col], errors='coerce').fillna(0.0)

# Agora pode aplicar o cálculo normalmente
bd_prod_nec['PCS_NEC_PROD_MESMA_REG_SIM_NAO'] = bd_prod_nec.apply(
    lambda row: (
        row['carteira_arraste_mes_anterior'] + row['carteira_mes_atual'] + row['saldo_previsao_pcs'] + row['SALDO_PREV_PROX_MES_PCS'] + row['estoq_seg_pcs']
        - (row['estoq_inicial_pcs'] + row['estoq_transf_pcs'])
    ) if row['tipo_produto'] == 'MR'
    else (
        row['carteira_arraste_mes_anterior'] + row['carteira_mes_atual'] + row['saldo_previsao_pcs'] + row['estoq_seg_pcs']
        - (row['estoq_inicial_pcs'] + row['estoq_transf_pcs'])
    ),
    axis=1
)

# Aplicar substituir null por Zero na coluna SALDO_PREV_PROX_MES_PCS
bd_prod_nec['SALDO_PREV_PROX_MES_PCS'] = bd_prod_nec['SALDO_PREV_PROX_MES_PCS'].fillna(0)

# Padronizando nomes de colunas para concluir bd_prod_nec
bd_prod_nec = bd_prod_nec.rename(columns=
    {'mes_ref': 'MES_REF', 
     'empresa': 'EMPRESA', 
     'cod_produto': 'COD_PROD', 
     'descricao': 'DESC_PROD',
     'linha_prod': 'LINHA_PROD',
     'familia_prod': 'FAMILIA_PROD',
     'tipo_produto': 'TIPO_PROD',
     'curva_abc': 'CURVA_ABC',
     'curva_123': 'CURVA_123',
     'estoq_seg_pcs': 'EST_SEG_PCS',
     'estoq_inicial_pcs': 'EST_INI_PCS',
     'carteira_arraste_mes_anterior': 'CART_ARR_MES_ANT',
     'carteira_mes_atual': 'CART_MES_ATUAL',
     'previsao_pcs': 'PREV_PCS',
     'saldo_previsao_pcs': 'SALDO_PREV_PCS',
     'peso_produto_kg': 'PESO_PROD_KG',
     'estoq_transf_pcs': 'EST_TRANS_PCS',
     'unidade_prod': 'UNID_PROD',
     'MESMA_REG': 'MESMA_REG',
     'saldo_previsao_pcs': 'SALDO_PREV_PCS',
     'lote_econ': 'LOTE_ECON',
     'qtd_emb': 'QTD_EMB',
     'lote_min': 'LOTE_MIN',
     }
)

# Definindo colunas e encerrando processo de formação da bd_prod_nec
bd_prod_nec = bd_prod_nec[
    ['MES_REF', 'EMPRESA', 'UNID_PROD', 'MESMA_REG', 'COD_PROD', 'DESC_PROD', 
     'LINHA_PROD', 'FAMILIA_PROD', 'TIPO_PROD', 'CURVA_ABC','CURVA_123', 'LOTE_ECON','QTD_EMB', 'LOTE_MIN', 
     'EST_SEG_PCS', 'EST_INI_PCS', 'CART_ARR_MES_ANT', 'CART_MES_ATUAL', 'PREV_PCS', 
     'SALDO_PREV_PCS', 'PESO_PROD_KG', 'EST_TRANS_PCS','PCS_NEC_PROD_MESMA_REG_SIM_NAO',
     'SALDO_PREV_PROX_MES_PCS']
].reset_index(drop=True)

In [4]:
### Transformação 04_GerarBaseLTP para gerar a base LTP

# Carregar base_dados_roteiros
bd_rot = pd.read_excel(
    pasta_input / 'base_dados_roteiros.xlsx',
    sheet_name='base_dados_roteiros',
    dtype={'cod_produto': str}
)

bd_rot['cod_produto'] = bd_rot['cod_produto'].astype(str).str.strip().str.upper()
bd_rot = bd_rot[bd_rot['cod_produto'].isin(produtos_param)].copy().reset_index(drop=True)

if 'mes_ref' in bd_rot.columns:
    bd_rot['mes_ref'] = pd.to_datetime(bd_rot['mes_ref'], errors='coerce', dayfirst=True)
    
colunas_manter = ['mes_ref', 'empresa', 'cod_produto', 'descricao', 'linha_prod', 'familia_prod', 'tipo_produto', 'cod_ferramenta', 'n_cavidade', 'ciclo_molde', 'pcs_hr', 'mo', 'alocacao_recurso', 'grupo_setor', 'prioridade']

bd_rot = bd_rot[colunas_manter].reset_index(drop=True)

# Adicionando nova coluna na tabela de roteiros
bd_rot['COD_FERR'] = (bd_rot['cod_ferramenta'] + "|" + bd_rot['empresa'].str[:3]).str.upper()

# Faz o merge trazendo somente as 4 colunas desejadas, associando pelos campos de chave
bd_rot = bd_rot.merge(
    bd_calend, 
    how='left', 
    left_on=['mes_ref', 'empresa'], 
    right_on=['MES_CALEND', 'UNIDADE']
)

# Removendo as colunas que não quero na bd_rot
colunas_excluir = ['MES_CALEND', 'UNIDADE']
bd_rot = drop_colunas(bd_rot, colunas_excluir)

# Carregando o Calendario de Recursos
bd_calend_rec = pd.read_excel(pasta_input / 'calend_recursos.xlsx', sheet_name='calend_recursos')

# Inserindo coluna PER_TURNO no calendário de recursos
bd_calend_rec['PER_TURNO'] = bd_calend_rec['turno_maq'] / 3

# Inserindo colunas do calendário de recursos na bd_rot
bd_rot = bd_rot.merge(
    bd_calend_rec, 
    how='left', 
    left_on=['mes_ref', 'empresa', 'alocacao_recurso'], 
    right_on=['mes_ref', 'unidade', 'maq']
)

colunas_excluir = ['unidade', 'maq', 'setores', 'turno_maq']
bd_rot = drop_colunas(bd_rot, colunas_excluir)

bd_rot.rename(columns={
    'revezamento': 'REC_revezamento',
    'horas_extras': 'REC_horas_extras',
    'disp_mes': 'REC_disp_mes',
    'horas_indisp': 'REC_horas_indisp',
    'PER_TURNO': 'REC_PER_TURNO'
}, inplace=True)

# Carregando Calendário de Ferramentas
bd_calend_ferr = pd.read_excel(pasta_input / 'calend_ferramentas.xlsx', sheet_name='calend_ferramentas')
bd_calend_ferr['NEW_MOLDES'] = bd_calend_ferr['moldes'].str.upper() + "|" + bd_calend_ferr['unidade'].str.upper()

# Merge entre bd_rot e bd_calend_ferr
bd_rot = bd_rot.merge(
    bd_calend_ferr, 
    how='left', 
    left_on=['mes_ref', 'empresa', 'cod_ferramenta'], 
    right_on=['mes_ref', 'unidade', 'moldes']
)

colunas_excluir = ['moldes','unidade', 'setores']
bd_rot = drop_colunas(bd_rot, colunas_excluir)

bd_rot.rename(columns={
    'horas_indisp': 'FER_horas_indisp',
    'disp_mes': 'FER_disp_mes',
    'NEW_MOLDES': 'moldes'
}, inplace=True)

# Preencher apenas os valores nulos de FER_horas_indisp com zero
bd_rot['FER_horas_indisp'] = bd_rot['FER_horas_indisp'].fillna(0)

# Calculando coluna HOR_REC_C1
# Se REC_revezamento for "SIM", calcula com TOT_HORAS_REV_CALEND, senão com TOT_HORAS_NOR_CALEND
# Calculando coluna HOR_REC_C1, se algum valor for nulo, retorna 0
bd_rot['HOR_REC_C1'] = np.where(
    bd_rot[['TOT_HORAS_REV_CALEND', 'REC_horas_extras', 'REC_horas_indisp', 'REC_PER_TURNO', 'REC_disp_mes', 'TOT_HORAS_NOR_CALEND']].isnull().any(axis=1),
    0,
    np.where(
        bd_rot['REC_revezamento'] == "SIM",
        ((bd_rot['TOT_HORAS_REV_CALEND'] + bd_rot['REC_horas_extras'] - bd_rot['REC_horas_indisp']) * bd_rot['REC_PER_TURNO']) * bd_rot['REC_disp_mes'],
        ((bd_rot['TOT_HORAS_NOR_CALEND'] + bd_rot['REC_horas_extras'] - bd_rot['REC_horas_indisp']) * bd_rot['REC_PER_TURNO']) * bd_rot['REC_disp_mes']
    )
).round(2)

# Calculando coluna HOR_FERR_C1
# Se REC_revezamento for "SIM", calcula com TOT_HORAS_REV_CALEND, senão com TOT_HORAS_NOR_CALEND
# Calculando coluna HOR_FER_C1, se algum valor for nulo, retorna 0, senão calcula conforme regra, com duas casas decimais
bd_rot['HOR_FER_C1'] = np.where(
    bd_rot[['TOT_HORAS_REV_CALEND', 'REC_horas_extras', 'FER_horas_indisp', 'TOT_HORAS_NOR_CALEND', 'FER_disp_mes']].isnull().any(axis=1),
    0,
    np.where(
        bd_rot['REC_revezamento'] == "SIM",
        (bd_rot['TOT_HORAS_REV_CALEND'] + (bd_rot['REC_horas_extras'] - bd_rot['FER_horas_indisp'])) * bd_rot['FER_disp_mes'],
        (bd_rot['TOT_HORAS_NOR_CALEND'] + (bd_rot['REC_horas_extras'] - bd_rot['FER_horas_indisp'])) * bd_rot['FER_disp_mes']
    )
).round(2)

# Criando coluna HOR_REC conforme a regra
# Se HOR_REC_C1 for null ou menor que 0, atribui 0; senão, atribui o valor de HOR_REC_C1
bd_rot['HOR_REC'] = np.where(
    bd_rot['HOR_REC_C1'].isnull() | (bd_rot['HOR_REC_C1'] < 0),
    0,
    bd_rot['HOR_REC_C1']
).round(2)

# Criando coluna HOR_FER conforme a regra
# Se HOR_FER_C1 for null ou menor que 0, atribui 0; senão, atribui o valor de HOR_FER_C1
bd_rot['HOR_FER'] = np.where(
    bd_rot['HOR_FER_C1'].isnull() | (bd_rot['HOR_FER_C1'] < 0),
    0,
    bd_rot['HOR_FER_C1']
).round(2)

# Criando coluna HOR_CAP conforme a regra
# Se HOR_REC for menor que HOR_FER, atribui HOR_REC; senão, atribui HOR_FER
bd_rot['HOR_CAP'] = np.where(
    bd_rot['HOR_REC'] < bd_rot['HOR_FER'],
    bd_rot['HOR_REC'],
    bd_rot['HOR_FER']
).round(2)

# Eliminando colunas para tornar mais clean a bd_rot
colunas_excluir = [
    'REC_revezamento', 'REC_horas_extras', 'REC_disp_mes', 'REC_horas_indisp',
    'TOT_HORAS_REV_CALEND', 'TOT_HORAS_NOR_CALEND', 'REC_PER_TURNO',
    'FER_horas_indisp', 'HOR_REC_C1', 'HOR_FER_C1', 'disp_mes'
]
bd_rot = drop_colunas(bd_rot, colunas_excluir)

# Renomeando a coluna prioridade do bd_rot para PRIORIDADE_ROTEIRO, para evitar confusão 
# com a prioridade da bd_mat_param
bd_rot.rename(columns={'prioridade': 'PRIORIDADE_ROTEIRO'}, inplace=True)

# Executando o merge entre bd_rot e bd_mat_param para trazer os campos unidade_fat e prioridade
bd_rot = bd_rot.merge(
    bd_mat_param, 
    how='left', 
    left_on=['empresa', 'cod_produto'], 
    right_on=['unidade_prod', 'produto']
)

# Elimnando campos que não são mais necessários da bd_rot
colunas_excluir = ['produto', 'unidade_prod', 'tipo_abast']
bd_rot = drop_colunas(bd_rot, colunas_excluir)

# Renomeando colunas da matriz parametros que foram trazidas para a bd_rot
bd_rot.rename(columns={
    'unidade_fat': 'MAT_PAR_unidade_fat',
    'prioridade': 'MAT_PAR_prioridade'
}, inplace=True)


# Executando o merge entre bd_rot e bd_prod_necs para trazer os campos de volumese e dados
bd_rot = bd_rot.merge(
    bd_prod_nec, 
    how='left', 
    left_on=['mes_ref', 'MAT_PAR_unidade_fat', 'cod_produto'], 
    right_on=['MES_REF', 'EMPRESA', 'COD_PROD']
)

# Elimnando campos que não são mais necessários da bd_rot
colunas_excluir = [
    'TOT_DIAS_NOR_CALEND','TOT_DIAS_REV_CALEND', 'moldes', 'MES_REF', 'EMPRESA',
    'UNID_PROD', 'COD_PROD', 'DESC_PROD', 'LINHA_PROD',
    'FAMILIA_PROD', 'TIPO_PROD', 'CURVA_ABC', 'CURVA_123'
]
bd_rot = drop_colunas(bd_rot, colunas_excluir)

# Renomeando da bd_roteiro que foram trazidas para a bd_rot
bd_rot.rename(columns={
    'mes_ref': 'MES_REF',
    'empresa': 'EMPRESA',
    'cod_produto': 'COD_PROD',
    'descricao': 'DESC_PROD',
    'linha_prod': 'LINHA_PROD',
    'familia_prod': 'FAMILIA_PROD',
    'tipo_produto': 'TIPO_PROD',
    'COD_FERR': 'COD_FER_UNID',
    'cod_ferramenta': 'COD_FERR',
    'n_cavidade': 'N_CAVIDADE',
    'ciclo_molde': 'CICLO_MOLDE',
    'pcs_hr': 'PCS_HORA',
    'mo': 'MO',
    'alocacao_recurso': 'ALOC_REC',
    'grupo_setor': 'GRUPO_SETOR',
    'peso_produto_kg': 'PESO_PROD',
    'PRIORIDADE_ROTEIRO': 'PRIOR_ROT',
    'MAT_PAR_unidade_fat': 'UNID_FAT_MATPAR',
    'MAT_PAR_prioridade': 'PRIOR_MATPAR'
}, inplace=True)

# Executando o merge entre bd_rot e bd_matriz_regioes para trazer os campo região fat
bd_rot = bd_rot.merge(
    bd_matriz_regioes, 
    how='left', 
    left_on=['UNID_FAT_MATPAR'], 
    right_on=['Unidade']
)

# Elimnando campos que não são mais necessários da bd_rot
bd_rot = drop_colunas(bd_rot, ['Unidade'])

# Renomeando da campos que foram trazidos para a bd_rot
bd_rot.rename(columns={'Região': 'REG_UNID_FAT'}, inplace=True)

# Executando o merge entre bd_rot e bd_matriz_regioes para trazer os campo região prod
bd_rot = bd_rot.merge(
    bd_matriz_regioes, 
    how='left', 
    left_on=['EMPRESA'], 
    right_on=['Unidade']
)

# Elimnando campos que não são mais necessários da bd_rot
bd_rot = drop_colunas(bd_rot, ['Unidade'])

# Renomeando da campos que foram trazidos para a bd_rot
bd_rot.rename(columns={'Região': 'REG_UNID_PROD'}, inplace=True)

# Criando o campo MESMA_REG conforme a regra solicitada
bd_rot['MESMA_REG'] = np.where(
    bd_rot['REG_UNID_PROD'].isna(),
    "NAO",
    np.where(
        bd_rot['REG_UNID_FAT'] == bd_rot['REG_UNID_PROD'],
        "SIM",
        "NAO"
    )
)

# Substituir null (NaN) por 0 nos campos solicitados
campos_zerar = [
    'EST_SEG_PCS', 'EST_INI_PCS',
    'CART_ARR_MES_ANT', 'CART_MES_ATUAL', 'PREV_PCS', 'SALDO_PREV_PCS',
    'PESO_PROD_KG', 'EST_TRANS_PCS', 'SALDO_PREV_PROX_MES_PCS', 'PCS_NEC_PROD_MESMA_REG_SIM_NAO'
]
bd_rot[campos_zerar] = bd_rot[campos_zerar].fillna(0)

# Criando a coluna MES_REF_ANT reduzindo 1 mês de MES_REF
bd_rot['MES_REF_ANT'] = (bd_rot['MES_REF'] - pd.DateOffset(months=1)).dt.to_period('M').dt.to_timestamp()

# Renomeando da bd_roteiro que foram trazidas para a bd_rot
bd_rot.rename(columns={
    'EMPRESA': 'UNID_PROD',
    'UNID_FAT_MATPAR': 'UNID_FAT',
}, inplace=True)

# Importar aba matriz_regioes da planilha matriz limitantes
bd_matriz_limitantes = pd.read_excel(pasta_input / 'matriz_limitantes.xlsx', sheet_name='matriz_limitantes', dtype={'produto': str})

if 'mes_ref' in bd_matriz_limitantes.columns:
    bd_matriz_limitantes['mes_ref'] = pd.to_datetime(bd_matriz_limitantes['mes_ref'], errors='coerce', dayfirst=True)

# Executando o merge entre bd_rot e bd_matriz_limitantes para trazer os campo limitante_pcs
bd_rot = bd_rot.merge(
    bd_matriz_limitantes, 
    how='left',
    left_on=['MES_REF','UNID_FAT','COD_PROD'],
    right_on=['mes_ref','unidade','produto']
)

# Elimnando campos que não são mais necessários da bd_rot
colunas_excluir = ['mes_ref','unidade','produto']
bd_rot = drop_colunas(bd_rot, colunas_excluir)

# Renomeando da campos que foram trazidos para a bd_rot
bd_rot.rename(columns={'limitante_pcs': 'LIMIT_PCS'}, inplace=True)

# Aplicando a regra no campo LIMIT_PCS conforme lógica Java
bd_rot['AVALIAR'] = bd_rot['PRIOR_MATPAR'].astype(str) + "|" + bd_rot['PRIOR_ROT'].astype(str)
bd_rot['LIMIT_PCS'] = np.where(
    (bd_rot['AVALIAR'] != "1|1") | (bd_rot['LIMIT_PCS'].isnull()),
    0,
    bd_rot['LIMIT_PCS']
)

# Formatando a coluna PRIOR_MATPAR para inteiro
bd_rot['PRIOR_MATPAR'] = bd_rot['PRIOR_MATPAR'].fillna(0).astype(int)

# Eliminar a coluna AVALIAR
colunas_excluir = ['AVALIAR', 'MES_REF_ANT']
bd_rot = drop_colunas(bd_rot, colunas_excluir)

# Criar ID_COMP
bd_rot['ID_COMP'] = (
    bd_rot['MES_REF'].dt.strftime('%b%y').str.upper() + "|" +
    bd_rot['UNID_PROD'].str.upper() + "|" +
    bd_rot['COD_PROD'].str.upper()
)

# Criar ID_RECURSO
bd_rot['ID_RECURSO'] = (
    bd_rot['MES_REF'].dt.strftime('%b%y').str.upper() + "|" +
    bd_rot['UNID_PROD'].str.upper() + "|" +
    bd_rot['ALOC_REC'].str.upper()
)

# Criar ID_FERRAMENTA
bd_rot['ID_FERRAMENTA'] = (
    bd_rot['MES_REF'].dt.strftime('%b%y').str.upper() + "|" +
    bd_rot['UNID_PROD'].str.upper() + "|" +
    bd_rot['COD_FER_UNID'].str.upper()
)

# Criar ID_PROD_UNID_FAT
bd_rot['ID_PROD_UNID_FAT'] = (
    bd_rot['MES_REF'].dt.strftime('%b%y').str.upper() + "|" +
    bd_rot['UNID_FAT'].str.upper() + "|" +
    bd_rot['COD_PROD'].astype(str).str.upper()
)

# Criar ID_PROD_UNID_FAT_ANT já calculando o mês anterior no mesmo comando
bd_rot['ID_PROD_UNID_FAT_ANT'] = (
    (bd_rot['MES_REF'] - pd.DateOffset(months=1)).dt.strftime('%b%y').str.upper() + "|" +
    bd_rot['UNID_FAT'].str.upper()  + "|" +
    bd_rot['COD_PROD'].str.upper()
)
 
# Criar a coluna IND
# Classificação antiga
# bd_rot = bd_rot.sort_values(by=['MES_REF', 'UNID_FAT', 'COD_PROD', 'PRIOR_MATPAR', 'PRIOR_ROT', 'TIPO_PROD']).reset_index(drop=True)

# Classificação nova
bd_rot = bd_rot.sort_values(by=['MES_REF', 'TIPO_PROD', 'UNID_FAT', 'COD_PROD', 'PRIOR_MATPAR', 'PRIOR_ROT']).reset_index(drop=True)
bd_rot['IND'] = range(1, len(bd_rot) + 1)

# |||||||||||||||||||||||||||||||| ID_NUM_REC ||||||||||||||||||||||||||||||||
# Remover linhas onde ID_RECURSO é NaN
bd_rot = bd_rot[bd_rot['ID_RECURSO'].notna()].copy()

# Ordena primeiro por ID_RECURSO e depois por IND
# bd_rot = bd_rot.sort_values(by=['ID_RECURSO', 'IND']).reset_index(drop=True)

# Cria o índice incremental por ID_RECURSO, começando em 1
# bd_rot['ID_NUM_REC'] = (bd_rot.groupby('ID_RECURSO').cumcount() + 1).astype(int)

# Classificar crescente pela coluna IND
bd_rot = bd_rot.sort_values(by=['IND']).reset_index(drop=True)
# |||||||||||||||||||||||||||||||| ID_NUM_REC ||||||||||||||||||||||||||||||||

# |||||||||||||||||||||||||||||||| ID_NUM_FER ||||||||||||||||||||||||||||||||
# Remover linhas onde ID_FERRAMENTA é NaN
bd_rot = bd_rot[bd_rot['ID_FERRAMENTA'].notna()].copy()

# # Ordena primeiro por ID_NUM_FER e depois por IND
# bd_rot = bd_rot.sort_values(by=['ID_FERRAMENTA', 'IND']).reset_index(drop=True)

# # Cria o índice incremental por ID_NUM_FER, começando em 1
# bd_rot['ID_NUM_FER'] = (bd_rot.groupby('ID_FERRAMENTA').cumcount() + 1).astype(int)

# Classificar crescente pela coluna IND
bd_rot = bd_rot.sort_values(by=['IND']).reset_index(drop=True)
# |||||||||||||||||||||||||||||||| ID_NUM_FER ||||||||||||||||||||||||||||||||

# Criar coluna ID_CONC_REC = ID_RECURSO + "|" + ID_NUM_REC e ID_CONC_FER = ID_FERRAMENTA + "|" + ID_NUM_FER
# bd_rot['ID_CONC_REC'] = bd_rot['ID_RECURSO'] + "|" + bd_rot['ID_NUM_REC'].astype(str)
# bd_rot['ID_CONC_FER'] = bd_rot['ID_FERRAMENTA'] + "|" + bd_rot['ID_NUM_FER'].astype(str)

# Organizando Layout Final da para dados para bd_base_LTP
# Ordenando colunas
nova_ordem_colunas = [
    'MES_REF', 'UNID_FAT', 'UNID_PROD', 'MESMA_REG', 'PRIOR_MATPAR', 'PRIOR_ROT', 'COD_PROD', 'DESC_PROD', 'LINHA_PROD',
    'FAMILIA_PROD', 'TIPO_PROD', 'COD_FER_UNID', 'N_CAVIDADE', 'CICLO_MOLDE', 'PCS_HORA', 'MO', 'ALOC_REC', 
    'GRUPO_SETOR', 'PESO_PROD_KG', 'LOTE_ECON', 'QTD_EMB','LOTE_MIN', 'IND', 'ID_COMP', 'ID_RECURSO', 'ID_FERRAMENTA', 'ID_PROD_UNID_FAT', 'ID_PROD_UNID_FAT_ANT', 'HOR_REC', 'HOR_FER', 'HOR_CAP', 'PREV_PCS', 'EST_SEG_PCS', 'EST_INI_PCS', 'CART_ARR_MES_ANT', 'CART_MES_ATUAL', 'SALDO_PREV_PCS', 'EST_TRANS_PCS', 'SALDO_PREV_PROX_MES_PCS', 'LIMIT_PCS'
    ]

# # Reordenando as colunas
bd_prod_rot = bd_rot[nova_ordem_colunas].reset_index(drop=True)

# Criar tabela com ID_PROD_UNID_FAT e MESMA_REG, filtrando PRIOR_MATPAR = 1 e PRIOR_ROT = 1
bd_prod_rot_PRIOR_11 = bd_prod_rot[
    (bd_prod_rot['PRIOR_MATPAR'] == 1) & (bd_prod_rot['PRIOR_ROT'] == 1)
][['ID_PROD_UNID_FAT', 'MESMA_REG']].drop_duplicates().reset_index(drop=True)
bd_prod_rot_PRIOR_11.rename(columns={'MESMA_REG': 'MESMA_REG_PRIOR_11'}, inplace=True)

# Fazer join entre bd_prod_rot e bd_prod_rot_PRIOR_11 para trazer MESMA_REG_PRIOR_11
bd_prod_rot = bd_prod_rot.merge(
    bd_prod_rot_PRIOR_11,
    how='left',
    on='ID_PROD_UNID_FAT'
)

# Substituir coluna MESMA_REG por MESMA_REG_PRIOR_11
bd_prod_rot['MESMA_REG'] = bd_prod_rot['MESMA_REG_PRIOR_11'].fillna(bd_prod_rot['MESMA_REG'])
# Eliminar coluna MESMA_REG_PRIOR_11
bd_prod_rot = drop_colunas(bd_prod_rot, ['MESMA_REG_PRIOR_11'])

# Criar bd_estrutura_filtrada, com base na bd_estruturas eliminando cod_insumo que não constam na coluna COD_PROD da bd_prod_rot
# codigos_validos = set(bd_prod_rot['COD_PROD'].unique())
# bd_estrutura_filtrada = bd_estruturas[bd_estruturas['cod_insumo'].isin(codigos_validos)].copy().reset_index(drop=True)

# Criar conjunto de tuplas válidas (COD_PROD, UNID_PROD)
codigos_validos = set(zip(bd_prod_rot['COD_PROD'], bd_prod_rot['UNID_PROD']))

# Filtrar estrutura com base em cod_insumo + empresa
bd_estrutura_filtrada = (
    bd_estruturas[
        bd_estruturas[['cod_insumo', 'empresa']].apply(tuple, axis=1).isin(codigos_validos)
    ]
    .copy()
    .reset_index(drop=True)
)

# Mesclar bd_base_LTP e bd_estrutura_filtrada por UNID_PROD, COD_PROD vs empresa e cod_prod_acabado
bd_prod_rot_estr = pd.merge(bd_prod_rot, bd_estrutura_filtrada, how='left', left_on=['UNID_PROD', 'COD_PROD'], right_on=['empresa', 'cod_prod_acabado']).reset_index(drop=True)

# Função para criar estrutura com fator estrutural
bd_estr_fator_estrutural = criar_estrutura_com_fator_estrutural(bd_estrutura_filtrada)

print("✅ Geração de tabelas de dados LTP concluída!")

✅ Geração de tabelas de dados LTP concluída!


In [5]:
########### TRAZER CAMPOS DA MATRIZ TRANSFERENCIA E ESTOQUE ORIGEM ###########
bd_LTP_NEC = bd_prod_rot.copy()

# Trazer para bd_produtos_nec as coluna de status da tabela bd_mat_transf_comp, colunas considera_estoq_origem e considera_estoq_triangulacao

 # Carregar matriz_transf_componentes
bd_mat_transf_comp = pd.read_excel(
    pasta_input / 'matriz_transf_componentes.xlsx',
    sheet_name='matriz_transf_componentes',
    dtype={'cod_produto': str}
)
bd_mat_transf_comp['cod_produto'] = bd_mat_transf_comp['cod_produto'].astype(str).str.strip().str.upper()
# Filtrar o DataFrame bd_mat_transf_comp para manter apenas as linhas onde 'validacao' == 'Sim'
bd_mat_transf_comp = bd_mat_transf_comp[bd_mat_transf_comp['validacao'].str.upper() == 'SIM']
bd_mat_transf_comp = bd_mat_transf_comp.reset_index(drop=True)

bd_LTP_NEC = bd_LTP_NEC.merge(
    bd_mat_transf_comp[['unid_destino', 'unid_origem', 'unid_triangulacao', 'cod_produto', 'considera_estoq_origem', 'considera_estoq_triangulacao']],
    how='left',
    left_on=['UNID_FAT', 'COD_PROD'],
    right_on=['unid_destino', 'cod_produto']
)

# Trazer os valores de Estoque Inicial e Estoque Transferencia das unidades Origem
# Cria uma cópia da tabela para servir como origem
bd_produtos_origem = bd_LTP_NEC[['UNID_FAT', 'MES_REF', 'COD_PROD', 'EST_INI_PCS', 'EST_TRANS_PCS']].copy()
bd_produtos_origem = bd_produtos_origem.rename(columns={
    'UNID_FAT': 'UNID_FAT_ORIGEM',
    'MES_REF': 'MES_REF_ORIGEM',
    'COD_PROD': 'COD_PROD_ORIGEM',
    'EST_INI_PCS': 'EST_INI_PCS_ORIGEM',
    'EST_TRANS_PCS': 'EST_TRANS_PCS_ORIGEM'
})

# Eliminar duplicatas da tabela de origem
bd_produtos_origem = bd_produtos_origem.drop_duplicates(subset=['UNID_FAT_ORIGEM', 'MES_REF_ORIGEM', 'COD_PROD_ORIGEM', 'EST_INI_PCS_ORIGEM', 'EST_TRANS_PCS_ORIGEM'])

# Faz o merge na própria tabela, buscando os valores da origem
bd_LTP_NEC = bd_LTP_NEC.merge(
    bd_produtos_origem,
    how='left',
    left_on=['unid_origem', 'MES_REF', 'COD_PROD'],
    right_on=['UNID_FAT_ORIGEM', 'MES_REF_ORIGEM', 'COD_PROD_ORIGEM']
)

# Criar uma tabela com cópia na bd_produtos_nec ter somente as colunas de interesse e buscar os valores de estoque origem e estoque triangulação
bd_produtos_estoque_origem_triangulacao = bd_LTP_NEC
colunas_excluir = ['considera_estoq_origem', 'considera_estoq_triangulacao']
bd_produtos_estoque_origem_triangulacao = drop_colunas(bd_produtos_estoque_origem_triangulacao, colunas_excluir)

# Trazer para bd_produtos_nec as colunas de status da tabela bd_mat_transf_comp, colunas considera_estoq_origem e considera_estoq_triangulacao
bd_produtos_estoque_origem_triangulacao = bd_produtos_estoque_origem_triangulacao.merge(
    bd_mat_transf_comp[['unid_origem', 'cod_produto', 'considera_estoq_origem']],
    how='left',
    left_on=['UNID_FAT', 'COD_PROD'],
    right_on=['unid_origem', 'cod_produto']
)

# Excluir as colunas unid_triangulacao e cod_produto
colunas_excluir = ['unid_triangulacao', 'cod_produto']
bd_produtos_estoque_origem_triangulacao = drop_colunas(bd_produtos_estoque_origem_triangulacao, colunas_excluir)

bd_produtos_estoque_origem_triangulacao = bd_produtos_estoque_origem_triangulacao.merge(
    bd_mat_transf_comp[['unid_triangulacao', 'cod_produto', 'considera_estoq_triangulacao']],
    how='left',
    left_on=['UNID_FAT', 'COD_PROD'],
    right_on=['unid_triangulacao', 'cod_produto']
)

# Excluir as colunas unid_destino e cod_produto
colunas_excluir = ['unid_triangulacao', 'cod_produto']
bd_produtos_estoque_origem_triangulacao = drop_colunas(bd_produtos_estoque_origem_triangulacao, colunas_excluir)

bd_produtos_estoque_origem_triangulacao = bd_produtos_estoque_origem_triangulacao[['MES_REF', 'UNID_FAT', 'COD_PROD', 'EST_INI_PCS', 'EST_TRANS_PCS','considera_estoq_origem', 'considera_estoq_triangulacao']]

bd_produtos_estoque_origem_triangulacao.reset_index(drop=True, inplace=True)

# Trazer os valores de Estoque Inicial e Estoque Transferencia das unidades Triangulação
# Cria uma cópia da tabela para servir como Triangulação
bd_produtos_triangulacao = bd_LTP_NEC[['UNID_FAT', 'MES_REF', 'COD_PROD', 'EST_INI_PCS', 'EST_TRANS_PCS']].copy()

bd_produtos_triangulacao = bd_produtos_triangulacao.rename(columns={
    'UNID_FAT': 'UNID_FAT_TRIANG',
    'MES_REF': 'MES_REF_TRIANG',
    'COD_PROD': 'COD_PROD_TRIANG',
    'EST_INI_PCS': 'EST_INI_PCS_TRIANG',
    'EST_TRANS_PCS': 'EST_TRANS_PCS_TRIANG'
})

# Faz o merge na própria tabela, buscando os valores da TRIANG
bd_LTP_NEC = bd_LTP_NEC.merge(
    bd_produtos_triangulacao,
    how='left',
    left_on=['unid_triangulacao', 'MES_REF', 'COD_PROD'],
    right_on=['UNID_FAT_TRIANG', 'MES_REF_TRIANG', 'COD_PROD_TRIANG']
)

# Somar valores Estoque Inicial e Transferencia da Origem
def ORI_TOT_PCS(row):
    if str(row.get('considera_estoq_origem', '')).strip().upper() == 'SIM':
        return row.get('EST_INI_PCS_ORIGEM', 0) + row.get('EST_TRANS_PCS_ORIGEM', 0)
    else:
        return 0
bd_LTP_NEC['ORI_TOT_PCS'] = bd_LTP_NEC.apply(ORI_TOT_PCS, axis=1)

# Somar valores Estoque Inicial e Transferencia da Triangulação
def TRIANG_TOT_PCS(row):
    if str(row.get('considera_estoq_triangulacao', '')).strip().upper() == 'SIM':
        return row.get('EST_INI_PCS_TRIANG', 0) + row.get('EST_TRANS_PCS_TRIANG', 0)
    else:
        return 0
bd_LTP_NEC['TRIANG_TOT_PCS'] = bd_LTP_NEC.apply(TRIANG_TOT_PCS, axis=1)

# Eliminar colunas desnecessárias da bd_LTP_NEC
colunas_excluir = [
    'unid_destino', 'unid_origem', 'unid_triangulacao', 'cod_produto',
    'considera_estoq_origem', 'considera_estoq_triangulacao',
    'UNID_FAT_ORIGEM', 'MES_REF_ORIGEM', 'COD_PROD_ORIGEM',
    'EST_INI_PCS_ORIGEM', 'EST_TRANS_PCS_ORIGEM',
    'UNID_FAT_TRIANG', 'MES_REF_TRIANG', 'COD_PROD_TRIANG',
    'EST_INI_PCS_TRIANG', 'EST_TRANS_PCS_TRIANG'
]
bd_LTP_NEC = drop_colunas(bd_LTP_NEC, colunas_excluir)

# Aplicar zero nos campos vazios de NEC_PCS, ORI_TOT_PCS e TRIANG_TOT_PCS
colunas_preencher = ['ORI_TOT_PCS', 'TRIANG_TOT_PCS']
bd_LTP_NEC[colunas_preencher] = bd_LTP_NEC[colunas_preencher].fillna(0).astype(float)

# Filtrar bd_LTP_NEC pelo campo MAT_PAR_prioridade > 0
bd_LTP_NEC = bd_LTP_NEC[bd_LTP_NEC['PRIOR_MATPAR'] > 0].reset_index(drop=True)

#*****************************************# ID_ULT_PRIORI #*****************************************
# Criar a tabela bd_Ultimo_Roteiro_MAT_PAR com os campos ID_PROD_UNID_FAT, PRIOR_MATPAR e PRIOR_ROT, eliminando as duplicatas
bd_Ultimo_Roteiro_MAT_PAR = bd_LTP_NEC[['ID_PROD_UNID_FAT', 'PRIOR_MATPAR', 'PRIOR_ROT']]
bd_Ultimo_Roteiro_MAT_PAR = bd_Ultimo_Roteiro_MAT_PAR.sort_values(
    by=['ID_PROD_UNID_FAT', 'PRIOR_MATPAR', 'PRIOR_ROT'],
    ascending=[True, True, False]
).reset_index(drop=True)
bd_Ultimo_Roteiro_MAT_PAR = bd_Ultimo_Roteiro_MAT_PAR.drop_duplicates(subset='ID_PROD_UNID_FAT', keep='first').reset_index(drop=True)

# Criar a coluna ID_ULT_PRIORI fazendo o merge entre bd_LTP_NEC e bd_Ultimo_Roteiro_MAT_PAR para trazer dados a coluna ID_PROD_UNID_FAT
bd_LTP_NEC = bd_LTP_NEC.merge(
    bd_Ultimo_Roteiro_MAT_PAR[['ID_PROD_UNID_FAT', 'PRIOR_MATPAR', 'PRIOR_ROT']].rename(columns={'ID_PROD_UNID_FAT': 'ID_ULT_PRIORI'}),
    how='left',
    left_on=['ID_PROD_UNID_FAT', 'PRIOR_MATPAR', 'PRIOR_ROT'],
    right_on=['ID_ULT_PRIORI', 'PRIOR_MATPAR', 'PRIOR_ROT']
)

bd_LTP_NEC = bd_LTP_NEC.sort_values(['IND']).reset_index(drop=True)

# Zerar valores campo SALDO_PREV_PROX_MES_PCS, considerando se MESMA_REG for igual a "SIM", manter valor, se MESMA_REG for "NAO", zerar o valor
bd_LTP_NEC['SALDO_PREV_PROX_MES_PCS'] = np.where(
    bd_LTP_NEC['MESMA_REG'] == 'NAO',
    bd_LTP_NEC['SALDO_PREV_PROX_MES_PCS'],
    0
)

# Criando cópia dos campos e adicinar nos rótulos, no incio do nome LTP
campos_copiar = [
    'EST_SEG_PCS', 'EST_INI_PCS', 'CART_ARR_MES_ANT', 'CART_MES_ATUAL', 'SALDO_PREV_PCS', 'EST_TRANS_PCS','SALDO_PREV_PROX_MES_PCS'
]

for col in campos_copiar:
    bd_LTP_NEC['LTP_' + col] = bd_LTP_NEC[col]
    
# Adicionar coluna LTP_COMP_NEC_PCS com valor 0
bd_LTP_NEC['LTP_COMP_NEC_PCS'] = 0

# Utilizando Flags Parametros Lote Minimo e Multiplo Embalagens e Calculando os campos VAR_NEC1, VAR_NEC2 e VAR_NEC3
def carregar_flags_ltp(pasta_painel):
    def carregar_planilha_segura(caminho: Path, **kwargs):
        try:
            return load_workbook(caminho, **kwargs)
        except PermissionError:
            with tempfile.NamedTemporaryFile(suffix=caminho.suffix, delete=False) as tmp:
                shutil.copy2(caminho, tmp.name)
                return load_workbook(tmp.name, **kwargs)

    def obter_valor_nome_definido(wb, nome_definido):
        nome_planilha, intervalo = next(nome_definido.destinations)
        planilha = wb[nome_planilha]
        coluna_min, linha_min, _, _ = range_boundaries(intervalo)
        return planilha.cell(linha_min, coluna_min).value

    arquivo_painel = pasta_painel / 'Painel_LTP.xlsm'
    planilha_ltp = carregar_planilha_segura(arquivo_painel, data_only=True)
    nome_lote_min = planilha_ltp.defined_names['FlagLoteMinimo']
    nome_multiplo_emb = planilha_ltp.defined_names['FlagMultiploEmb']
    lote_min = obter_valor_nome_definido(planilha_ltp, nome_lote_min)
    multiplo_emb = obter_valor_nome_definido(planilha_ltp, nome_multiplo_emb)
    return str(lote_min).strip().upper(), str(multiplo_emb).strip().upper()

lote_min_flag, multiplo_emb_flag = carregar_flags_ltp(pasta_painel)

# Mover Colunas ID_RECURSO e id_FERRAMENTA para o final do DataFrame, depois da coluna LTP_COMP_NEC_PCS
bd_LTP_NEC = bd_LTP_NEC[
    [col for col in bd_LTP_NEC.columns if col not in ['ID_RECURSO', 'ID_FERRAMENTA']] +
    ['ID_RECURSO', 'ID_FERRAMENTA']
]

# Mover colunas HOR_REC, HOR_FER, HOR_CAP para o final do DataFrame, depois da coluna LTP_COMP_NEC_PCS
bd_LTP_NEC = bd_LTP_NEC[
    [col for col in bd_LTP_NEC.columns if col not in ['HOR_REC', 'HOR_FER']] +
    ['HOR_REC', 'HOR_FER']
]

# Excluir coluna HOR_CAP
bd_LTP_NEC = drop_colunas(bd_LTP_NEC, ['HOR_CAP'])

# criar coluna COMP com valor 'SIM' ou 'NAO' dependendo se o COD_PROD está na tabela bd_estruturas
insumos = set(bd_estruturas['cod_insumo'].astype(str).str.strip().str.upper())
bd_LTP_NEC['COMP'] = (bd_LTP_NEC['COD_PROD'].astype(str).str.strip().str.upper().isin(insumos).map({True: 'SIM', False: 'NAO'}))

print("✅ Primeiro Calculo NEC_PCS e Distribuição Capacidade concluídos!")

✅ Primeiro Calculo NEC_PCS e Distribuição Capacidade concluídos!


In [ ]:
# Normalizar Data
bd_LTP_NEC['MES_REF'] = bd_LTP_NEC['MES_REF'].dt.normalize()

# Criar uma lista de MES_REF únicos
mes_refs = bd_LTP_NEC['MES_REF'].unique().tolist()

# Loop sobre os meses (no momento apenas o primeiro é tratado com cálculo)
for mes_ref in mes_refs:
    # Filtrar bd_LTP_M1 por mes_ref
    if mes_ref == mes_refs[0]:
        bd_LTP_M1 = bd_LTP_NEC[bd_LTP_NEC['MES_REF'] == mes_ref].reset_index(drop=True)
        bd_LTP_M1 = calc_nec_pcs_hr(bd_LTP_M1, lote_min_flag, multiplo_emb_flag)
        bd_LTP_M1, tab_HOR_REC, tab_HOR_FER = calcular_distrib_capacidade(bd_LTP_M1, lote_min_flag, multiplo_emb_flag)
        bd_LTP_M1, tab_NEC_N_ATEND_PCS, tab_NEC_ESTOURO_PCS, tab_NEC_N_ATEND_PCS_REC_FER = calcular_demais_campos(bd_LTP_M1)
        # # *************************# Explosão da Estrutura e necessidade de componentes #**************************
        bd_ltp_estrutura_explodida = explodir_estrutura_ltp(bd_estrutura_filtrada, bd_LTP_M1)
        bd_nec_comp_expl= calcular_explosao_necessidades(bd_ltp_estrutura_explodida, bd_LTP_M1, lote_min_flag, multiplo_emb_flag)
        # *************************# Retorno da NEC_COMP_PCS para a estrutura LTP #********************************
        # Agregar campos UNID_PROD, COD_INSUMO, NEC_LIQ_PCS
        bd_nec_comp_expl_agreg = (bd_nec_comp_expl.groupby(["UNID_PROD", "COD_INSUMO"], as_index=False)["NEC_LIQ_PCS"].sum())
        
        # Faz o merge entre bd_LTP_M1 e bd_nec_comp_expl_agreg
        bd_LTP_M1 = bd_LTP_M1.merge(
            bd_nec_comp_expl_agreg,
            left_on=["UNID_FAT", "COD_PROD"],
            right_on=["UNID_PROD", "COD_INSUMO"],
            how="left",
            suffixes=("", "_eliminar")
        )

        # Atualiza o campo LTP_COMP_NEC_PCS com NEC_LIQ_PCS quando existir
        bd_LTP_M1["LTP_COMP_NEC_PCS"] = bd_LTP_M1["NEC_LIQ_PCS"].fillna(bd_LTP_M1["LTP_COMP_NEC_PCS"])

        # Remove coluna auxiliar COD_INSUMO se não precisar mais
        bd_LTP_M1 = bd_LTP_M1.drop(columns=["COD_INSUMO", "NEC_LIQ_PCS", "UNID_PROD_eliminar"])
        # *********************# Recalculando NEC_PCS e demais campos, após explosão componentes #*******************************
        bd_LTP_M1 = calc_nec_pcs_hr(bd_LTP_M1, lote_min_flag, multiplo_emb_flag)
        bd_LTP_M1, tab_HOR_REC, tab_HOR_FER = calcular_distrib_capacidade(bd_LTP_M1, lote_min_flag, multiplo_emb_flag)
        bd_LTP_M1, tab_NEC_N_ATEND_PCS, tab_NEC_ESTOURO_PCS, tab_NEC_N_ATEND_PCS_REC_FER = calcular_demais_campos(bd_LTP_M1)
        # **************************************# Iniciando Cortes Ferramentas #*************************************************
        def aplicar_cortes_recursos(bd_LTP):
            
            bd_mat_cortes = cria_bd_mat_cortes_REC(bd_LTP)
            
            # Coluna Total Estoque para otimizar e reduzir tamanho dos próximos calculos que debitam estoque
            ET_PCS = bd_LTP_M1['LTP_EST_INI_PCS'] + bd_LTP_M1['LTP_EST_TRANS_PCS'] + bd_LTP_M1['TRIANG_TOT_PCS'] + bd_LTP_M1['ORI_TOT_PCS']
            
            bd_LTP_M1['ET_PCS'] = np.where(
                bd_LTP_M1['NEC_PCS'] == 0,
                0,
                ET_PCS
            )
            
            # Calculos identificando quantidades PCS não cobertas por estoque e que devem ser cortadas
            C_ARR_PCS = (bd_LTP['LTP_CART_ARR_MES_ANT'] - bd_LTP['ET_PCS']).clip(lower=0)
            
            bd_LTP['C_ARR_PCS'] = np.where(
                bd_LTP_M1['NEC_PCS'] == 0,
                0,
                C_ARR_PCS
            )
            
            C_AT_PCS = ((bd_LTP['LTP_CART_ARR_MES_ANT'] + bd_LTP['LTP_CART_MES_ATUAL']) - bd_LTP['ET_PCS']).clip(lower=0) - bd_LTP['C_ARR_PCS']
            
            bd_LTP['C_AT_PCS'] = np.where(
                bd_LTP_M1['NEC_PCS'] == 0,
                0,
                C_AT_PCS
            )
            
            PV_PCS  = ((bd_LTP['LTP_CART_ARR_MES_ANT'] + bd_LTP['LTP_CART_MES_ATUAL'] + bd_LTP['LTP_SALDO_PREV_PCS']) - bd_LTP['ET_PCS']).clip(lower=0) - (bd_LTP['C_ARR_PCS'] + bd_LTP['C_AT_PCS'])
            
            bd_LTP['PV_PCS'] = np.where(
                bd_LTP_M1['NEC_PCS'] == 0,
                0,
                PV_PCS
            )
            
            PV_PROX_PCS = np.where(bd_LTP['MESMA_REG'] == 'NAO', ((bd_LTP['LTP_CART_ARR_MES_ANT'] + bd_LTP['LTP_CART_MES_ATUAL'] + bd_LTP['LTP_SALDO_PREV_PCS'] + bd_LTP['LTP_SALDO_PREV_PROX_MES_PCS']) - bd_LTP['ET_PCS']).clip(lower=0) - (bd_LTP['C_ARR_PCS'] + bd_LTP['C_AT_PCS'] + bd_LTP['PV_PCS']), 0)
            
            bd_LTP['PV_PROX_PCS'] = np.where(
                bd_LTP_M1['NEC_PCS'] == 0,
                0,
                PV_PROX_PCS
            )

            bd_LTP['ES_PCS'] = np.where(
                bd_LTP['NEC_PCS'] == 0, 0,
                np.where(
                    bd_LTP['MESMA_REG'] == 'NAO',
                    np.maximum(
                        (bd_LTP['LTP_CART_ARR_MES_ANT'] + bd_LTP['LTP_CART_MES_ATUAL'] +
                        bd_LTP['LTP_SALDO_PREV_PCS'] + bd_LTP['LTP_SALDO_PREV_PROX_MES_PCS'] +
                        bd_LTP['LTP_EST_SEG_PCS']) - bd_LTP['ET_PCS'] -
                        (bd_LTP['C_ARR_PCS'] + bd_LTP['C_AT_PCS'] +
                        bd_LTP['PV_PCS'] + bd_LTP['PV_PROX_PCS']), 0),
                    np.maximum(
                        (bd_LTP['LTP_CART_ARR_MES_ANT'] + bd_LTP['LTP_CART_MES_ATUAL'] +
                        bd_LTP['LTP_SALDO_PREV_PCS'] + bd_LTP['LTP_EST_SEG_PCS']) -
                        bd_LTP['ET_PCS'] -
                        (bd_LTP['C_ARR_PCS'] + bd_LTP['C_AT_PCS'] +
                        bd_LTP['PV_PCS'] + bd_LTP['PV_PROX_PCS']), 0)
                )
            )
            
            DIF_LM_PCS = bd_LTP['NEC_PCS'] - (bd_LTP['C_ARR_PCS'] + bd_LTP['C_AT_PCS'] + bd_LTP['PV_PCS'] + bd_LTP['PV_PROX_PCS'] + bd_LTP['ES_PCS'])
            bd_LTP['DIF_LM_PCS'] = np.where(
                bd_LTP['NEC_PCS'] <= 0,
                0,
                np.where(
                    DIF_LM_PCS > bd_LTP['LOTE_MIN'],
                    bd_LTP['LOTE_MIN'],
                    DIF_LM_PCS
                )
            )
            
            DIF_EMB_PCS = bd_LTP['NEC_PCS'] - (bd_LTP['C_ARR_PCS'] + bd_LTP['C_AT_PCS'] + bd_LTP['PV_PCS'] + bd_LTP['PV_PROX_PCS'] + bd_LTP['ES_PCS'] + bd_LTP['DIF_LM_PCS'])
            
            bd_LTP['DIF_EMB_PCS'] = np.where(
                bd_LTP['NEC_PCS'] <= 0,
                0,
                DIF_EMB_PCS
            )
            
            # Tranformar em HR as colunas calculadas em PCS para cortes
            bd_LTP['C_ARR_HR'] = bd_LTP['C_ARR_PCS'] / bd_LTP['PCS_HORA']
            bd_LTP['C_AT_HR'] = bd_LTP['C_AT_PCS'] / bd_LTP['PCS_HORA']
            bd_LTP['PV_HR'] = bd_LTP['PV_PCS'] / bd_LTP['PCS_HORA']
            bd_LTP['PV_PROX_HR'] = bd_LTP['PV_PROX_PCS'] / bd_LTP['PCS_HORA']
            bd_LTP['ES_HR'] = bd_LTP['ES_PCS'] / bd_LTP['PCS_HORA']
            bd_LTP['DIF_LM_HR'] = bd_LTP['DIF_LM_PCS'] / bd_LTP['PCS_HORA']
            bd_LTP['DIF_EMB_HR'] = bd_LTP['DIF_EMB_PCS'] / bd_LTP['PCS_HORA']
            
            # FIXME: FILTRAR bd_mat_cortes campo ALOC_REC
            bd_mat_cortes = bd_mat_cortes[bd_mat_cortes['ALOC_REC'] == '00618'].reset_index(drop=True)
            
            # Iniciar lopp de cortes por recurso
            for i in range(len(bd_mat_cortes)):
                aloc_rec = bd_mat_cortes.at[i, 'ALOC_REC']
                unid_prod = bd_mat_cortes.at[i, 'UNID_PROD']

                # Filtrar bd_LTP para o recurso atual
                filtro = (bd_LTP['ALOC_REC'] == aloc_rec) & (bd_LTP['UNID_PROD'] == unid_prod)
                bd_LTP_filtrado = bd_LTP.loc[filtro].copy()
                
                # Somar NEC_PCS com TIPO_PROD = 'PI', para validar se entrar no tratamento de PI ou se segue para tratamento de PA
                soma_nec_pcs_pi = bd_LTP_filtrado[bd_LTP_filtrado['TIPO_PROD'] == 'PI']['NEC_PCS'].sum()
                
                # Verificar se em bd_LTP_filtrado existe TIPO_PROD = PI, e se soma_nec_pcs_pi > 0
                if 'PI' in bd_LTP_filtrado['TIPO_PROD'].values and soma_nec_pcs_pi > 0:
                    # Listar os COD_PROD e UNID_PROD que sejam TIPO_PROD = "PI", eliminando duplicatas, eliminando NEC_PCS = 0
                    pi_itens = bd_LTP_filtrado[bd_LTP_filtrado['TIPO_PROD'] == 'PI'][['COD_PROD', 'UNID_PROD', 'PCS_HORA', 'NEC_PCS']].drop_duplicates().reset_index(drop=True)
                    pi_itens = pi_itens[pi_itens['NEC_PCS'] > 0].reset_index(drop=True)
                    pi_itens['RASTREABILIDADE'] = pi_itens['COD_PROD'] + '|' + pi_itens['UNID_PROD']
                    pi_itens['COD_INSUMO'] = pi_itens['COD_PROD']
                    # Filtrar COD_PROD e UNID_PROD do pi_itens, nos campos COD_INSUMO e UNID_PROD da bd_ltp_estrutura_explodida
                    lista_itens_pa_estr_expl = bd_ltp_estrutura_explodida[
                        (bd_ltp_estrutura_explodida[['COD_INSUMO', 'UNID_PROD']].apply(tuple, axis=1).isin(
                            pi_itens[['COD_PROD', 'UNID_PROD']].apply(tuple, axis=1)
                        ))][['COD_PROD', 'UNID_PROD']].drop_duplicates().reset_index(drop=True)

                    # Filtrar cada COD_PROD e UNID_PROD da lista_itens_pa_estr_expl na bd_LTP_filtrado, na bd_LTP_M1, trazendo as linhas encontradas incoporporando em bd_LTP_filtrado, mantendo o índice original  
                    for _, pi in pi_itens.iterrows():
                        cod_pi = pi['COD_PROD']
                        unid_pi = pi['UNID_PROD']
                        rastreabilidade = pi['RASTREABILIDADE']
                        pcs_hora_pi = pi['PCS_HORA']
                        cod_insumo = pi['COD_INSUMO']

                        # PAs originados por esse PI na estrutura
                        lista_itens_pa_estr_expl = bd_ltp_estrutura_explodida[
                            (bd_ltp_estrutura_explodida['COD_INSUMO'] == cod_pi) &
                            (bd_ltp_estrutura_explodida['UNID_PROD'] == unid_pi)
                        ][['COD_PROD', 'UNID_PROD']].drop_duplicates()

                        for _, pa in lista_itens_pa_estr_expl.iterrows():
                            cod_pa = pa['COD_PROD']
                            unid_prod_estrutura = pa['UNID_PROD']

                            filtro_estrutura = (
                                (bd_LTP['COD_PROD'] == cod_pa) &
                                (bd_LTP['UNID_PROD'] == unid_prod_estrutura)
                            )

                            linhas_filtradas_pa_LTP = bd_LTP.loc[filtro_estrutura].copy()

                            linhas_filtradas_pa_LTP['RASTREABILIDADE'] = rastreabilidade
                            linhas_filtradas_pa_LTP['PCS_HORA_PI'] = pcs_hora_pi
                            linhas_filtradas_pa_LTP['COD_INSUMO'] = cod_insumo

                            bd_LTP_filtrado = pd.concat(
                                [bd_LTP_filtrado, linhas_filtradas_pa_LTP],
                                ignore_index=False
                            )
                            
                    # Trazer para bd_LTP_filtrado a coluna FATOR_ESTRUTURAL da bd_estr_fator_estrutural, pelas colunas COD_PROD e UNID_PROD e COD_INSUMO
                    bd_LTP_filtrado = bd_LTP_filtrado.merge(
                        bd_estr_fator_estrutural[['COD_PROD', 'UNID_PROD', 'COD_INSUMO', 'FATOR_ESTRUTURAL']],
                        how='left',
                        left_on=['COD_PROD', 'UNID_PROD', 'COD_INSUMO'],
                        right_on=['COD_PROD', 'UNID_PROD', 'COD_INSUMO']
                    )
                    
                    # Criar coluna NEC_ATEND_PCS_PI, calculando NEC_ATEND_PCS multiplicando por FATOR_ESTRUTURAL
                    bd_LTP_filtrado['NEC_ATEND_PCS_PI'] = bd_LTP_filtrado['NEC_ATEND_PCS'] * bd_LTP_filtrado['FATOR_ESTRUTURAL']
                    
                    # Criar coluna NEC_ATEND_HR_PI, calculando NEC_ATEND_PCS_PI dividindo por PCS_HORA_PI
                    bd_LTP_filtrado['NEC_ATEND_HR_PI'] = bd_LTP_filtrado['NEC_ATEND_PCS_PI'] / bd_LTP_filtrado['PCS_HORA_PI']
                    
                    # ----------------------------------------- Realizar Rateio ---------------------------------------------
                    # Distribuir NEC_ATEND_HR_PI nas colunas C_ARR_HR, C_AT_HR, PV_HR, PV_PROX_HR, ES_HR, DIF_LM_HR, DIF_EMB_HR proporcionalmente
                    cols_rateio = ['C_ARR_HR','C_AT_HR','PV_HR','PV_PROX_HR','ES_HR','DIF_LM_HR','DIF_EMB_HR']

                    # 1. Copiar base original
                    base = bd_LTP_filtrado[cols_rateio].copy()

                    # 2. Calcular base de rateio
                    base_sum = base.sum(axis=1)

                    # 3. Evitar divisão por zero
                    fator = bd_LTP_filtrado['NEC_ATEND_HR_PI'].where(base_sum > 0, 0)

                    # 4. Rateio e sobreescrever as colunas originais com os valores rateados
                    bd_LTP_filtrado[cols_rateio] = base.div(base_sum, axis=0).mul(fator, axis=0).fillna(0)
                    # --------------------------------------- Fim Realizar Rateio -------------------------------------------
                    
                    # Loop sob a bd_mat_cortes para identificar o corte CORTE_HR e aplicar na bd_LTP_filtrado, conforme matriz de cortes e recursos, filtros de mesma região e recurso, e coluna de referencia da matriz de cortes, para identificar os produtos que devem ser cortados, e as horas a serem cortadas
                    for j in range(len(bd_mat_cortes)):
                        aloc_rec_corte = bd_mat_cortes.at[j, 'ALOC_REC']
                        unid_prod_corte = bd_mat_cortes.at[j, 'UNID_PROD']
                        corte_hr = bd_mat_cortes.at[j, 'CORTE_HR']
                        coluna_ref_corte = bd_mat_cortes.at[j, 'COLUNA_REF']

                        # Filtrar bd_LTP_filtrado para o recurso e unidade de produto do corte
                        filtro_corte = (bd_LTP_filtrado['ALOC_REC'] == aloc_rec_corte) & (bd_LTP_filtrado['UNID_PROD'] == unid_prod_corte)
                        bd_LTP_filtrado_corte = bd_LTP_filtrado.loc[filtro_corte].copy()

                        # Aplicar o corte_hr na coluna_ref_corte da bd_LTP_filtrado_corte, reduzindo o valor da coluna_ref_corte pelo corte_hr, sem reduzir abaixo de zero
                        bd_LTP_filtrado.loc[filtro_corte, coluna_ref_corte] = (bd_LTP_filtrado.loc[filtro_corte, coluna_ref_corte] - corte_hr).clip(lower=0)
                    
                    
                    # Função que cria a matriz de cortes, e sequencia de cortes que deve ser respeitada
                    bd_matriz_cortes_rec = matriz_cortes_horas(bd_LTP_filtrado)
                    
                    

                    '''
                        Filtrar mesma região matriz e coluna de analise no BD_FILTRADO
                        IR na coluna especifica e calcular partipação no total, de cada produto, na coluna foco da matriz de cortes
                        
                    '''
                    
                else:
                    # Aplicar cortes pois no filtro máquina temos somente PAs
                    pass
            return  bd_LTP, bd_mat_cortes, bd_LTP_filtrado, pi_itens, linhas_filtradas_pa_LTP, bd_estr_fator_estrutural, bd_matriz_cortes_rec

    bd_LTP_M1, bd_mat_cortes, bd_LTP_filtrado, pi_itens, linhas_filtradas_pa_LTP, bd_estr_fator_estrutural, bd_matriz_cortes_rec = aplicar_cortes_recursos(bd_LTP_M1)

In [15]:
bd_mat_cortes

,UNID_PROD,ALOC_REC,HOR_REC,NEC_ESTOURO_HR_REC,NEC_ATEND_HR,%_OCUP_REC,CORTE_HR
0,Matriz,00618,511.52,289.19,511.52,156.54,289.19


In [12]:
# Exportar para Excel

# Listar os dataframes e seus nomes correspondentes
dataframes_para_exportar = {
    "bd_LTP_M1": bd_LTP_M1,
    "bd_mat_cortes": bd_mat_cortes,
    "bd_estrutura_filtrada": bd_estrutura_filtrada,
    "bd_nec_comp_expl": bd_nec_comp_expl,
    "bd_LTP_filtrado": bd_LTP_filtrado,
    "bd_estr_fator_estrutural": bd_estr_fator_estrutural
}

for nome in dataframes_para_exportar:
    df = globals()[nome]   # pega o objeto DataFrame pelo nome da variável
    caminho_arquivo = pasta_output / f"{nome}.xlsx"
    df.to_excel(caminho_arquivo, index=False)
    print(f"✅ Exportado: {caminho_arquivo}")
    
timer.finalizar()
print("🎯 Processo concluído com sucesso!")

✅ Exportado: c:\Users\carlo\OneDrive\BC\03. Projetos Bedin\01. Krona\LTP\02_OUTPUT\bd_LTP_M1.xlsx
✅ Exportado: c:\Users\carlo\OneDrive\BC\03. Projetos Bedin\01. Krona\LTP\02_OUTPUT\bd_mat_cortes.xlsx
✅ Exportado: c:\Users\carlo\OneDrive\BC\03. Projetos Bedin\01. Krona\LTP\02_OUTPUT\bd_estrutura_filtrada.xlsx
✅ Exportado: c:\Users\carlo\OneDrive\BC\03. Projetos Bedin\01. Krona\LTP\02_OUTPUT\bd_nec_comp_expl.xlsx
✅ Exportado: c:\Users\carlo\OneDrive\BC\03. Projetos Bedin\01. Krona\LTP\02_OUTPUT\bd_LTP_filtrado.xlsx
✅ Exportado: c:\Users\carlo\OneDrive\BC\03. Projetos Bedin\01. Krona\LTP\02_OUTPUT\bd_estr_fator_estrutural.xlsx
Tempo total de processamento: 28 min 47.2 s
🎯 Processo concluído com sucesso!
